In [1]:
import os
import random
import soundfile as sf
from faster_whisper import WhisperModel
from pydub import AudioSegment

/root/anaconda3/envs/whisper_env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
import string

In [3]:
# Load model globally (only once)
MODEL_SIZE = "large-v3"
LANGUAGE = "ur"
DEVICE = "cuda"

In [4]:
print("🔄 Loading Whisper model...")
model = WhisperModel(MODEL_SIZE, device=DEVICE)
print("✅ Whisper model loaded.")

🔄 Loading Whisper model...
✅ Whisper model loaded.


In [5]:
def generate_random_string(length=12):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

In [6]:
def transcribe_and_chunk_for_tts(input_path, output_dir):
    # Generate random folder suffix
    random_suffix = generate_random_string()
    input_basename = os.path.splitext(os.path.basename(input_path))[0]
    unique_folder_name = f"{input_basename}_{random_suffix}"

    # Create directories
    output_dir = os.path.join(output_dir, unique_folder_name)
    os.makedirs(output_dir, exist_ok=True)

    audio_chunks_dir = os.path.join(output_dir, "chunks")
    os.makedirs(audio_chunks_dir, exist_ok=True)

    metadata_path = os.path.join(output_dir, f"{input_basename}_metadata.csv")

    # Convert audio to WAV at 48kHz mono
    temp_wav_path = os.path.join(output_dir, f"{input_basename}_converted.wav")
    audio = AudioSegment.from_file(input_path)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(temp_wav_path, format="wav")

    # Transcribe
    segments, info = model.transcribe(
        temp_wav_path,
        language=LANGUAGE,
        beam_size=5,
        vad_filter=True,
        vad_parameters={"min_silence_duration_ms": 2000}
    )

    # Load audio for slicing
    full_audio = AudioSegment.from_wav(temp_wav_path)

    # Save chunks and metadata
    metadata = []
    for i, segment in enumerate(segments):
        start_ms = int(segment.start * 1000)
        end_ms = int(segment.end * 1000)
        chunk_audio = full_audio[start_ms:end_ms]

        # Random suffix for chunk filename
        chunk_rand = generate_random_string(4)
        chunk_filename = f"{input_basename}_chunk_{i:04d}_{int(segment.end * 1000)}ms_{chunk_rand}.wav"
        chunk_path = os.path.join(audio_chunks_dir, chunk_filename)

        # Export at 48kHz mono
        chunk_audio = chunk_audio.set_frame_rate(22500).set_channels(1)
        chunk_audio.export(chunk_path, format="wav")

        # Sanitize and add to metadata
        text = segment.text.strip().replace("\n", " ").replace("|", "").replace('"', '')
        # print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {text}")
        metadata.append(f"{chunk_filename},{text}")

    # Write metadata CSV
    with open(metadata_path, "w", encoding="utf-8") as f:
        f.write("filename,text\n")
        for line in metadata:
            f.write(line + "\n")

    print(f"✅ Saved {len(metadata)} chunks to {audio_chunks_dir}")
    print(f"📝 Metadata saved to {metadata_path}")


In [7]:
# transcribe_and_chunk_for_tts('vocal_1/mdx_extra_q/ooi0itvn_1743338606/vocals.mp3', 'tts_dataset')

In [8]:
import glob

In [16]:
vocal_0 = glob.glob('vocals/mdx_extra_q/*/vocals.mp3')

In [10]:
def process_audio_files(file_list, output_root_dir):
    print(f"🎧 Starting transcription of {len(file_list)} file(s)...\n")

    success_count = 0
    failed_files = []

    for idx, file_path in enumerate(file_list, start=1):
        print(f"🔹 [{idx}/{len(file_list)}] Processing: {file_path}")
        try:
            transcribe_and_chunk_for_tts(file_path, output_root_dir)
            print(f"✅ Successfully processed: {file_path}\n")
            success_count += 1
        except Exception as e:
            print(f"❌ Failed to process: {file_path}")
            print(f"   🧨 Error: {e}\n")
            failed_files.append(file_path)

    # Summary
    print("📝 Processing complete.")
    print(f"✔️  {success_count} succeeded")
    print(f"❌ {len(failed_files)} failed")

    if failed_files:
        print("\n🚨 Failed files:")
        for f in failed_files:
            print(f"   - {f}")


In [ ]:
process_audio_files(vocal_0, 'tts_dataset')

🎧 Starting transcription of 30 file(s)...

🔹 [1/30] Processing: vocals/mdx_extra_q/5xgedse2_1743337471/vocals.mp3


In [17]:
vocal_1 = glob.glob('vocal_1/mdx_extra_q/*/vocals.mp3')

In [ ]:
process_audio_files(vocal_1, 'tts_dataset')

🎧 Starting transcription of 245 file(s)...

🔹 [1/245] Processing: vocal_1/mdx_extra_q/dzfa4kex_1743334767/vocals.mp3
✅ Saved 327 chunks to tts_dataset/vocals_xm2zreejoou6/chunks
📝 Metadata saved to tts_dataset/vocals_xm2zreejoou6/vocals_metadata.csv
✅ Successfully processed: vocal_1/mdx_extra_q/dzfa4kex_1743334767/vocals.mp3

🔹 [2/245] Processing: vocal_1/mdx_extra_q/b7uzt4q2_1743319749/vocals.mp3
✅ Saved 265 chunks to tts_dataset/vocals_2l83g2l471y8/chunks
📝 Metadata saved to tts_dataset/vocals_2l83g2l471y8/vocals_metadata.csv
✅ Successfully processed: vocal_1/mdx_extra_q/b7uzt4q2_1743319749/vocals.mp3

🔹 [3/245] Processing: vocal_1/mdx_extra_q/v2fxi61e_1743336332/vocals.mp3
✅ Saved 181 chunks to tts_dataset/vocals_egup146jjt06/chunks
📝 Metadata saved to tts_dataset/vocals_egup146jjt06/vocals_metadata.csv
✅ Successfully processed: vocal_1/mdx_extra_q/v2fxi61e_1743336332/vocals.mp3

🔹 [4/245] Processing: vocal_1/mdx_extra_q/tnpkilgi_1743336478/vocals.mp3
✅ Saved 369 chunks to tts_datas

In [ ]:
print('DONE')

In [ ]:
vocal_2 = glob.glob('vocal_2/mdx_extra_q/*/vocals.mp3')

In [ ]:
process_audio_files(vocal_2, 'tts_dataset')

In [22]:
print('Done')

Done
